<a id="Title"></a>


In [2]:
#always run "source ~/anaconda3/bin/activate root" before running anaconda-navigator to open this notebook

# LAB NOTEBOOK
## PROF. ANDREW C.R. MARTIN
## Project: CDR H3 Modelling Quality Prediction
### By: Lilian Denzler, Msci Student

#  

<a id="Introduction"></a>

### Introduction

Antibody modelling has come a far way in the recent decade. A great achievement has been the development of many sophisticated software programmes, such as "abYmod" written by Prof. Martin at UCL. 

- recent developments in the field
- problems i.e. CDRH3 modelling
- why modelling is so hard (biological background)
- quality assesment methods
- how this is different

<b>Project aim:</b>
The aim is to have by the end of the project: 

<b>Plan:</b>

1) Dataset creation
- AbDb pdb data
- sequence extraction
- antibody modelling dataset
- template dataset

2) RMS deviation calculations

3) Feature set creation
- sequence identity of template scores
- length
- hydrophobicity score
-...

4) Feature encoding

5) ML models

6) Evaluation

7) stand-alone web-programme


### Contents

[Title](#Title)

[Introduction](#Introduction)

[Setup and Organization](#SetupandOrganization)

[1. Dataset Generation](#DatasetGeneration)

[1.1 Antibody Sequence Parsing](#AntibodySequenceParsing)

[1.2 Antibody Modelling](#AntibodyModelling)

[1.3 RMSD Calculation](#RMSDCalculation)

<a id="SetupandOrganization"></a>

### Setup and Organisation

This lab-notebook (written in Jupyter-notebook, running a bash kernel) will include be used to document the workflow of this project. In order to ensure reproducibility of results, bash shell commands will be included in the following and all used employed programs are to be found on github (https://github.com/LilianDenzler/CDR_H3_quality/tree/master)

<i>Calender Week 40 & 41</i>

After discussing the options for computer setup with Prof. Martin, I decided to opt for running a Linux OS (Ubuntu 20.04.1 LTS) in a Virtual Machine on my Windows-Laptop. I used VirtualBox (https://www.virtualbox.org/) for this and booted the VM from an external hard-drive. This enabled me to use the BASH shell without any compliations, and also gave me additional storage for any large files I didn't want to save on my laptop. 

Afterwards, I set up Git in order to track changes. I also set up a GitHub account (User: LilianDenzler) and made a new repository titled "CDR_H3_quality". Then, I synchronized Git with Github. The directory <i>/git</i> contains all files that are pushed to the repository. Prof. Martin showed me how to log into his personal server remotely, which had abYmod, bioptools and csv2arff installed on it. 


<i>Calender Week 42</i>

Unison was set up to sync files between the remote server and my local client. This is important for backing up data and enables me to work with large datasets, which my laptop does not have sufficient computing power for. The synced data is saved in the <i> /sync_project</i> directory. Files in the <i> /NOSYNC</i> subdirectory will not be synced.

<a id="DatasetGeneration"></a>

## 1. Dataset Generation

The first step is to produce a dataset of Antibody models, which can then be examined regarding their quality in following steps.

![alt: Workflow for Dataset creation](Dataset_creation.png "Organisation of Datasets, Scripts, Documents, etc.")

<a id="AntibodySequenceParsing"></a>

### 1.1 Antibody Sequence Parsing

<i>Calender Week 42 & 43</i>

The Chothia-numbered non-redundant Antibody stucture AbDb-dataset from bioinf.org.uk/abs/abdb was downloaded. The sequence of these Antibodies had to be extracted from these PDB files. To acomplish this, I wrote a short python script ("seq_parser.py"), which returned the sequence in the following abYmod-compatible format:

L1 D  
L2 I                 
L3 Q        
L4 M                 
...    
H1 F      
H2 M      
H3 T      
H4 Q     
...     

In [1]:
python3 seq_parser.py ~/sync_project/actual_PDBs_NR ~/sync_project/input_Abymod_seq
#argv[1]= direcoty with actual PDBs, argv[2] = directory to keep input sequences

<a id="AbYmodModelling"></a>

### 1.2 AbYmod Modelling 

However, some of the antibodies in my newly-made dataset had redundant antibodies in the complete, redundant AbDb-dataset which is used by abYmod. I thus downloaded the textfile of PDB codes of redundant structures (also on bioinf.org.uk/abs/abdb). A python script formatted this into a dictionary. This dictionary will be used during the abYmod modelling step: For all input sequences it will be used to check if there are any redundant structures in the dataset, which will then be excluded along with the original PDB structure. This is important, as otherwise the modelling software will have an exact template and thus achieve 100% accuracy in modelling.  

The get_abYmod2.py passes a command to abYmod and saves the modelled PDB file under "<PDB_ID>.pdb.model". During the modelling process the redundant structures are excluded as templates. The template files abYmod creates during modelling (these contain,among other things, the PDB codes of the best structural template used for modelling for the CDRH3 loop and are thus of importance)are saved in a separate subdirectory as "<PDB_ID>.tpl".


In [3]:
nohup python3 get_abYmod2.py /home/lilian/sync_project/Redundant_PDBs.txt /home/lilian/sync_project/NOSYNC/input_Abymod_seq/ /home/lilian/sync_project/NOSYNC/abYmod_structures/ &>output_abymod.log &
#argv[1]=Redundant_PDB file; argv[2]=directory of .seq files; argv[3]=directory to save model.pdb files;
#[1] 2683019 Process ID (in case you need to kill process)
#use "nohup" so command can continue running in the background after you log out
#use "&" so it runs in background


[1] 14572
[1]+  Exit 1                  nohup python3 get_abYmod2.py /home/lilian/sync_project/Redundant_PDBs.txt /home/lilian/sync_project/NOSYNC/input_Abymod_seq/ /home/lilian/sync_project/NOSYNC/abYmod_structures/ &> output_abymod.log


<a id="RMSDCalculation"></a>

### 1.3 RMSD Calculation

In order the make a judgement on the accuracy of the generated model, we will not calculate the Root Mean Square Deviation values (actal structures vs. modelled structures). 

ProFit (http://www.bioinf.org.uk/software/profit/) is used to make these calculations. The atom-to-atom RMS as well as the C-alpha atom to C-alpha atom of each residue is calculated. 
The C-alpha-atom values give a better indication of the validity of the modelled overall backbone structure. The atom values give additional information on how well the side-chains are modelled.

To determine whether the loop conformation itself is modelled incorrectly, a local calculation is performed. Here, the CDRH3 loops of the actual structure and the model are fitted onto each other.
To determine whether the loop positioning is modelled incorrectly, a global calculation is performed. The entire structure is fitted for this. 


MAKE PYMOL GRAPH TO VISUALIZE WHAT THE FITTING MEANS


In [5]:
#nohup python3 save_RMS ~/sync_project/programs/RMS_calc ~/sync_project/actual_PDBs_NR ~/sync_project/NOSYNC/abYmod_structures &


We now have a single .csv file with all RMSD values for each antibody in our database. Additionally, we have a file conatining the "by_residue" RMSD values for each structure. 